In [59]:
import pandas as pd
from os.path import join
import re

In [3]:
data_path = join('dataset',  'wine_cleared.csv')

In [7]:
data = pd.read_csv(data_path, index_col='Unnamed: 0')

In [8]:
data.head()

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [10]:
data['price_round'] = data.price.round().astype('int')

Синтаксис регулярных выражений необычен и содержит различные символы, знаки препинания, математических операций и так далее. Подробнее ознакомиться с синтаксисом регулярных выражений поможет статья <a href="https://tproger.ru/translations/regular-expression-python/">Python RegEx: практическое применение регулярок</a>.

In [37]:
regex = '\d{4}'
data['year'] = data.title.str.findall(regex).str.get(0).astype('float') # Применяем float, чтобы не споткнуться на nan
data.year.isna().sum()

4609

Проверим, насколько надежен такой способ определения года производства вина.

Для начала взглянем на пропущенные значения:

In [33]:
data[data.year.isna()].title

63                Roland Champion NV Brut Rosé  (Champagne)
69                         Collet NV Brut Rosé  (Champagne)
237       Consorzio Vini Tipici di San Marino NV Moscato...
315       Bellussi NV Extra Dry  (Prosecco di Valdobbiad...
326       Marsuret NV Extra Dry  (Prosecco di Valdobbiad...
                                ...                        
129710         Molliver Vineyards NV Chambourcin (Virginia)
129719    Bruno Gobillard NV Domaine Vieilles Vignes Bru...
129743               Castelnau NV Réserve Brut  (Champagne)
129869       Biltmore Estate NV Brut Sparkling (California)
129929    Col Vetoraz Spumanti NV  Prosecco Superiore di...
Name: title, Length: 4609, dtype: object

Видим, что в таких наименованиях действительно отсутствуют числовые данные, в том числе похожие на год.

Гипотеза: вина, в названиях которых не указан год производства только что произведены. Поэтому заполнить пропуски можно будет максимальным значением  year.

Посмотрим на минимальное полученное значение года производства вина

In [36]:
data.year.min()

1000.0

Наблюдаем явную чушь. Посмотрим на то, как выглядят названия вин с годом производства меньше 1500-го:

In [34]:
data[data.year < 1500].title

3929             Don Cristobal 1492 2010 1492 Red (Mendoza)
16076     Don Cristobal 1492 2010 Finca La Niña Malbec (...
30001     1000 Stories 2013 Bourbon Barrel Aged Zinfande...
33110     Don Cristobal 1492 2010 Finca La Niña Cabernet...
48702     Don Cristobal 1492 2010 Finca La Niña Malbec (...
53228          1070 Green 2011 Sauvignon Blanc (Rutherford)
77996     1000 Stories 2014 Bourbon Barrel Aged Batch No...
82746        Cristobal 1492 2009 1492 White White (Mendoza)
83509                     Ikal 1150 2007 Malbec (Tupungato)
94395      Cristobal 1492 2007 Cabernet Sauvignon (Mendoza)
127114                Ikal 1150 2007 Chardonnay (Tupungato)
Name: title, dtype: object

Видим, что групп цифр в таких наименованиях несколько. Гипотеза: из групп по 4 цифры год обозначает максимальная. Проверим её, посмотрев на максимальное значение year.

In [38]:
data.year.max()

7200.0

И тут тоже наблюдаем явную чушь. 7200 год еще не скоро, а машину времени еще не изобрели. Посмотрим на названия вин для годов из будущего:

In [39]:
data[data.year > 2020].title

5346      Foxen 7200 2012 7200 Cabernet Sauvignon (Happy...
51320     Foxen 7200 2007 Vogelzang Vineyard Cabernet Sa...
67294     Foxen 7200 2012 Range 30 West Red (Happy Canyo...
77155     Foxen 7200 2015 Vogelzang Vineyard Sauvignon B...
89464     Foxen 7200 2012 7200 Grassini Family Vineyard ...
92740     Foxen 7200 2014 Vogelzang Vineyard Cabernet Sa...
95719     Foxen 7200 2012 Range 30 West Red (Happy Canyo...
97223     Foxen 7200 2012 7200 Vogelzang Vineyard Cabern...
99984     Foxen 7200 2008 Grassini Family Vineyard Caber...
101858    Foxen 7200 2007 Grassini Family Vineyard Caber...
101859    Foxen 7200 2008 Range 30 West Red (Happy Canyo...
105885             3000 BC 2009 Chenin Blanc (Stellenbosch)
108703    Foxen 7200 2014 Grassini Family Vineyard Caber...
109157    Foxen 7200 2013 7200 Vogelzang Vineyard Sauvig...
120445    Foxen 7200 2013 Vogelzang Vineyard Cabernet Sa...
Name: title, dtype: object

Проверить, насколько точно работает ваше регулярное выражение, а также найти реализацию популярных регулярных выражений (например, поиска номера телефона), вы сможете на сайте https://ihateregex.io.

Гипотеза не подтвердилась. Зато видим, что в таких названиях значения для определения года производства присутствуют.

Предлагаемый алгоритм выделения года:
1. Если групп из четырёх цифр вообще нет, то оставляем None, чтобы в дальнейшем заполнить максимальным значением года;
2. Если группа одна, проверяем её на вхождение в диапазон 1990 - 2020. Если не входит, возвращаем None;
3. Если групп несколько, берем ту, что входит в диапазон 1990-2020 и имеет максимальное значение.

In [74]:
regex = '\d{4}'
data[data.year > 2020].title.str.findall(regex)#.str #.get(0).astype('float') # Применяем float, чтобы не споткнуться на nan

5346      [7200, 2012, 7200]
51320           [7200, 2007]
67294           [7200, 2012]
77155           [7200, 2015]
89464     [7200, 2012, 7200]
92740           [7200, 2014]
95719           [7200, 2012]
97223     [7200, 2012, 7200]
99984           [7200, 2008]
101858          [7200, 2007]
101859          [7200, 2008]
105885          [3000, 2009]
108703          [7200, 2014]
109157    [7200, 2013, 7200]
120445          [7200, 2013]
Name: title, dtype: object

In [76]:
data.title.iloc[5346]

'Foxen 7200 2012 7200 Cabernet Sauvignon (Happy Canyon of Santa Barbara)'

In [71]:
def get_year(title: str) -> float:
    """
    Returns year from title of wines dataset like: 'Antichi Vinai 1877 2013 Pietralava Red (Etna)'

    Args:
        title (str): Title

    Returns:
        float: year
    """
    
    groups = re.findall(r'\d{4}', title)
    
    for group in groups:
        year = float(group)
        if 1990 <= year <= 2022:
            return year

    return None

In [78]:
data['year'] = data.title.map(get_year)

In [15]:
data['is_usa'] = (data.country == 'US').astype(int)

In [17]:
data.is_usa.head()

0    0
1    0
2    1
3    1
4    1
Name: is_usa, dtype: int64

### Задание 2.1
1 point possible (graded)

Выберите из списка две самых популярных (помимо США) страны, производящих вино.

In [18]:
data.country.value_counts()

US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14
Serbia  

### Задание 2.2
2 points possible (graded)

Создайте бинарные признаки is_france, is_italy наподобие признака is_usa.

В ответ впишите результат выполнения кода data['is_france'].sum():

In [20]:
data['is_france'] = (data.country == 'France').astype(int)
data.is_france.sum()

22093

В ответ впишите результат выполнения кода data['is_italy'].sum():

In [21]:
data['is_italy'] = (data.country == 'Italy').astype(int)
data.is_italy.sum()

19540

### Задание 2.3
1 point possible (graded)

Создайте новый бинарный признак old_wine, где значение 1 будет обозначать, что вино старше 2010 года.

В ответ впишите результат выполнения кода data['old_wine'].sum():

In [89]:
max_year = data.year.max()
data.year.fillna(max_year, inplace=True)

In [90]:
data.year = data.year.astype('int')

In [91]:
data['old_wine'] = (data.year < 2010).astype('int')
data.old_wine.head()

0    0
1    0
2    0
3    0
4    0
Name: old_wine, dtype: int64

In [92]:
data.old_wine.sum()

39605

Итак, правильный ответ с учетом правильного выделения года производства из title: 39605

In [99]:
re.findall(r'\(\w+?\s?\w+?\s?\w+?\s?\w+?\s?\w+\)', 'Antichi Vinai 1877 2013 Pietralava Red (Etna Mallini Bandito Znamenito)')

['(Etna Mallini Bandito Znamenito)']

In [107]:
def get_locality(title: str) -> str:
    """
    Returns locality from title.
    get_valley('Antichi Vinai 1877 2013 Pietralava Red (Etna Mallini Bandito Znamenito)')
    will return Etna Mallini Bandito Znamenito

    Args:
        title (str): Wine title

    Returns:
        str: Locality or None
    """
    
    return found[0][1:-1] if (found := re.findall(r'\(.*?\)', title)) else None

In [109]:
get_locality('Antichi Vinai 1877 2013 Pietralava Red (Etna Mallini Bandito Znamenito)')

'Etna Mallini Bandito Znamenito'

In [113]:
data['locality'] = data.title.map(get_locality)
data.locality.isna().sum()

10522

In [114]:
data.title[data.locality.isna()]

65                            Simonnet-Febvre 2015  Chablis
66        Vignerons des Terres Secrètes 2015  Mâcon-Mill...
95                               Henry Fessy 2015  Juliénas
96                                 Henry Fessy 2015  Régnié
117                                Rideau 2014 Estate Syrah
                                ...                        
129875         Domaine des Nugues 2015  Beaujolais-Villages
129892             La Mannella 2012  Brunello di Montalcino
129900                                   Psagot 2014 Merlot
129929    Col Vetoraz Spumanti NV  Prosecco Superiore di...
129931            Henri de Villamont 2005  Grands-Echezeaux
Name: title, Length: 10522, dtype: object